In [1]:
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
stops = set(stopwords.words('english'))

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train = pd.read_csv('train.csv')
print(len(train))
train[0:10]

19579


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
5,id22965,"A youth passed in solitude, my best years spen...",MWS
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP
7,id13515,The surcingle hung in ribands from my body.,EAP
8,id19322,I knew that you could not say to yourself 'ste...,EAP
9,id00912,I confess that neither the structure of langua...,MWS


In [3]:
test = pd.read_csv('test.csv')
print(len(test))
test[0:10]

8392


,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...
5,id27337,"""The thick and peculiar mist, or smoke, which ..."
6,id24265,"That which is not matter, is not at all unless..."
7,id25917,I sought for repose although I did not hope fo...
8,id04951,"Upon the fourth day of the assassination, a pa..."
9,id14549,"""The tone metaphysical is also a good one."


In [4]:
label_enconder = preprocessing.LabelEncoder()
label_enconder.fit(train['author'])
train['label_encoded'] = label_enconder.transform(train['author'])
train[0:10]

,id,text,author,label_encoded
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1
5,id22965,"A youth passed in solitude, my best years spen...",MWS,2
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP,0
7,id13515,The surcingle hung in ribands from my body.,EAP,0
8,id19322,I knew that you could not say to yourself 'ste...,EAP,0
9,id00912,I confess that neither the structure of langua...,MWS,2


## Pre-processing text

In [5]:
def transformText(text):
    
    stops = set(stopwords.words("english"))
    
    # Convert text to lower
    text = text.lower()
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Removing all the stopwords
    filtered_words = [word for word in text.split() if word not in stops]
   # filtered_words = [word for word in text.split()]

    # Removing all the tokens with lesser than 3 characters
    filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=3)
    
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    
    # Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Stemming
    text = gensim.parsing.preprocessing.stem_text(text)
    return text

In [6]:
train['text_processed']=train['text'].apply(lambda x: transformText(x))
train

,id,text,author,label_encoded,text_processed
0,id26305,"This process, however, afforded me no means of...",EAP,0,process howev afford mean ascertain dimens dun...
1,id17569,It never once occurred to me that the fumbling...,HPL,1,never occur fumbl might mere mistak
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0,left hand gold snuff box which caper hill cut ...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2,love spring look windsor terrac sixteen fertil...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1,find noth els even gold superintend abandon at...
5,id22965,"A youth passed in solitude, my best years spen...",MWS,2,youth pass solitud best year spent gentl femin...
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP,0,astronom perhap point took refug suggest non l...
7,id13515,The surcingle hung in ribands from my body.,EAP,0,surcingl hung riband bodi
8,id19322,I knew that you could not say to yourself 'ste...,EAP,0,knew could sai stereotomi without brought thin...
9,id00912,I confess that neither the structure of langua...,MWS,2,confess neither structur languag code govern p...


## Train_test split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train['text_processed'], train['label_encoded'], test_size = 0.2, random_state = 4)
true_label = np.array(y_test)
print("#" * 20 + " Some stats " + "#"*20)
print("Dataset training: {} uterances".format(x_train.shape[0]))
print("Dataset testing: {} uterances".format(x_test.shape[0]))
print("Different classes: {}".format(len(y_train.unique())))

#################### Some stats ####################
Dataset training: 15663 uterances
Dataset testing: 3916 uterances
Different classes: 3


In [8]:
features_2 = TfidfVectorizer(max_df = 0.9, ngram_range = (1,2), norm = 'l2')

In [9]:
x_train_features_2 = features_2.fit_transform(x_train)
x_test_features_2 = features_2.transform(x_test)

In [10]:
model_sgd_2 = SGDClassifier(loss = 'modified_huber', penalty = 'l2',)
model_sgd_2.fit(x_train_features_2,y_train)

/home/ubuntu/Envs/nlp/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', max_iter=5,
       n_iter=None, n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [11]:
preds_sgd_2 = model_sgd_2.predict(x_test_features_2)
preds_sgd_2_proba = model_sgd_2.predict_proba(x_test_features_2)

print("Current Accuracy: {0:.3f}".format(accuracy_score(preds_sgd_2,true_label)))
print("Log loss for this classifier {}".format(log_loss(true_label,preds_sgd_2_proba)))

Current Accuracy: 0.834
Log loss for this classifier 0.6683578540565921


In [12]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [13]:
log_loss(true_label,preds_sgd_2_proba)

0.66835785405659209

In [14]:
x_train.head()

9493    twice upon ventur express total incredul respe...
1919                      fire among crowd women children
1692    thought heard rat partit even paid littl atten...
2714                              it mere typograph error
9863    uttermost step led dread chamber larg fragment...
Name: text_processed, dtype: object

In [15]:
## model 3
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [16]:
tfv.fit(list(x_train) + list(x_test))
x_train_tfv =  tfv.transform(x_train) 
x_test_tfv = tfv.transform(x_test)
# -----
clf = LogisticRegression(C=1.0)
clf.fit(x_train_tfv, y_train)
predictions = clf.predict_proba(x_test_tfv)
print("logloss: %0.3f " % multiclass_logloss(true_label, predictions))

logloss: 0.598 


In [17]:
## Model 4 - testing with XGBoost
import xgboost as xgb

/home/ubuntu/Envs/nlp/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [44]:
clf = xgb.XGBClassifier(max_depth=4, n_estimators=3000, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
eval_set = [(x_test_tfv.tocsc(),y_test)]
clf.fit(x_train_tfv.tocsc(), y_train, eval_metric = "mlogloss", eval_set=eval_set, verbose=True)
preds_proba = clf.predict_proba(x_test_tfv.tocsc())
preds = classi.predict(x_test_tfv.tocsc())

[0]	validation_0-mlogloss:1.08939
[1]	validation_0-mlogloss:1.08112
[2]	validation_0-mlogloss:1.07378
[3]	validation_0-mlogloss:1.06719
[4]	validation_0-mlogloss:1.06148
[5]	validation_0-mlogloss:1.05596
[6]	validation_0-mlogloss:1.05113
[7]	validation_0-mlogloss:1.04619
[8]	validation_0-mlogloss:1.04202
[9]	validation_0-mlogloss:1.0379
[10]	validation_0-mlogloss:1.03374
[11]	validation_0-mlogloss:1.02965
[12]	validation_0-mlogloss:1.02607
[13]	validation_0-mlogloss:1.02238
[14]	validation_0-mlogloss:1.01923
[15]	validation_0-mlogloss:1.01625
[16]	validation_0-mlogloss:1.0134
[17]	validation_0-mlogloss:1.01049
[18]	validation_0-mlogloss:1.00751
[19]	validation_0-mlogloss:1.00475
[20]	validation_0-mlogloss:1.00161
[21]	validation_0-mlogloss:0.999105
[22]	validation_0-mlogloss:0.996441
[23]	validation_0-mlogloss:0.994243
[24]	validation_0-mlogloss:0.991691
[25]	validation_0-mlogloss:0.989269
[26]	validation_0-mlogloss:0.986922
[27]	validation_0-mlogloss:0.984395
[28]	validation_0-mloglos

[226]	validation_0-mlogloss:0.795971
[227]	validation_0-mlogloss:0.795503
[228]	validation_0-mlogloss:0.795133
[229]	validation_0-mlogloss:0.794577
[230]	validation_0-mlogloss:0.794186
[231]	validation_0-mlogloss:0.793666
[232]	validation_0-mlogloss:0.793108
[233]	validation_0-mlogloss:0.792634
[234]	validation_0-mlogloss:0.792182
[235]	validation_0-mlogloss:0.791694
[236]	validation_0-mlogloss:0.791256
[237]	validation_0-mlogloss:0.790792
[238]	validation_0-mlogloss:0.790334
[239]	validation_0-mlogloss:0.789885
[240]	validation_0-mlogloss:0.789602
[241]	validation_0-mlogloss:0.789218
[242]	validation_0-mlogloss:0.788654
[243]	validation_0-mlogloss:0.788132
[244]	validation_0-mlogloss:0.787712
[245]	validation_0-mlogloss:0.787281
[246]	validation_0-mlogloss:0.786915
[247]	validation_0-mlogloss:0.786527
[248]	validation_0-mlogloss:0.786004
[249]	validation_0-mlogloss:0.785566
[250]	validation_0-mlogloss:0.785404
[251]	validation_0-mlogloss:0.785007
[252]	validation_0-mlogloss:0.784465
[

[449]	validation_0-mlogloss:0.719651
[450]	validation_0-mlogloss:0.719386
[451]	validation_0-mlogloss:0.719133
[452]	validation_0-mlogloss:0.719021
[453]	validation_0-mlogloss:0.718729
[454]	validation_0-mlogloss:0.718447
[455]	validation_0-mlogloss:0.718199
[456]	validation_0-mlogloss:0.717964
[457]	validation_0-mlogloss:0.717644
[458]	validation_0-mlogloss:0.717321
[459]	validation_0-mlogloss:0.717049
[460]	validation_0-mlogloss:0.716748
[461]	validation_0-mlogloss:0.716467
[462]	validation_0-mlogloss:0.716172
[463]	validation_0-mlogloss:0.715776
[464]	validation_0-mlogloss:0.715478
[465]	validation_0-mlogloss:0.715219
[466]	validation_0-mlogloss:0.714942
[467]	validation_0-mlogloss:0.714668
[468]	validation_0-mlogloss:0.714385
[469]	validation_0-mlogloss:0.714076
[470]	validation_0-mlogloss:0.713885
[471]	validation_0-mlogloss:0.713818
[472]	validation_0-mlogloss:0.713501
[473]	validation_0-mlogloss:0.7132
[474]	validation_0-mlogloss:0.712821
[475]	validation_0-mlogloss:0.712441
[47

[671]	validation_0-mlogloss:0.673081
[672]	validation_0-mlogloss:0.673005
[673]	validation_0-mlogloss:0.672884
[674]	validation_0-mlogloss:0.672641
[675]	validation_0-mlogloss:0.6725
[676]	validation_0-mlogloss:0.672361
[677]	validation_0-mlogloss:0.672222
[678]	validation_0-mlogloss:0.672062
[679]	validation_0-mlogloss:0.67191
[680]	validation_0-mlogloss:0.671662
[681]	validation_0-mlogloss:0.671572
[682]	validation_0-mlogloss:0.671475
[683]	validation_0-mlogloss:0.671333
[684]	validation_0-mlogloss:0.671155
[685]	validation_0-mlogloss:0.670906
[686]	validation_0-mlogloss:0.670716
[687]	validation_0-mlogloss:0.67052
[688]	validation_0-mlogloss:0.670347
[689]	validation_0-mlogloss:0.670196
[690]	validation_0-mlogloss:0.670011
[691]	validation_0-mlogloss:0.669953
[692]	validation_0-mlogloss:0.669793
[693]	validation_0-mlogloss:0.669597
[694]	validation_0-mlogloss:0.669507
[695]	validation_0-mlogloss:0.669244
[696]	validation_0-mlogloss:0.669045
[697]	validation_0-mlogloss:0.668981
[698]

[893]	validation_0-mlogloss:0.642774
[894]	validation_0-mlogloss:0.642698
[895]	validation_0-mlogloss:0.642559
[896]	validation_0-mlogloss:0.642346
[897]	validation_0-mlogloss:0.642206
[898]	validation_0-mlogloss:0.642127
[899]	validation_0-mlogloss:0.641985
[900]	validation_0-mlogloss:0.642013
[901]	validation_0-mlogloss:0.64197
[902]	validation_0-mlogloss:0.641897
[903]	validation_0-mlogloss:0.641776
[904]	validation_0-mlogloss:0.641718
[905]	validation_0-mlogloss:0.641568
[906]	validation_0-mlogloss:0.641491
[907]	validation_0-mlogloss:0.641434
[908]	validation_0-mlogloss:0.641365
[909]	validation_0-mlogloss:0.641071
[910]	validation_0-mlogloss:0.641087
[911]	validation_0-mlogloss:0.640956
[912]	validation_0-mlogloss:0.640916
[913]	validation_0-mlogloss:0.640747
[914]	validation_0-mlogloss:0.640703
[915]	validation_0-mlogloss:0.640569
[916]	validation_0-mlogloss:0.640421
[917]	validation_0-mlogloss:0.640331
[918]	validation_0-mlogloss:0.64021
[919]	validation_0-mlogloss:0.64012
[920

...

[2198]	validation_0-mlogloss:0.573644
[2199]	validation_0-mlogloss:0.573587
[2200]	validation_0-mlogloss:0.573599
[2201]	validation_0-mlogloss:0.573581
[2202]	validation_0-mlogloss:0.573549
[2203]	validation_0-mlogloss:0.573507
[2204]	validation_0-mlogloss:0.573455
[2205]	validation_0-mlogloss:0.573459
[2206]	validation_0-mlogloss:0.573465
[2207]	validation_0-mlogloss:0.573519
[2208]	validation_0-mlogloss:0.573511
[2209]	validation_0-mlogloss:0.573438
[2210]	validation_0-mlogloss:0.573341
[2211]	validation_0-mlogloss:0.573339
[2212]	validation_0-mlogloss:0.573269
[2213]	validation_0-mlogloss:0.573281
[2214]	validation_0-mlogloss:0.57321
[2215]	validation_0-mlogloss:0.573113
[2216]	validation_0-mlogloss:0.573059
[2217]	validation_0-mlogloss:0.573018
[2218]	validation_0-mlogloss:0.573012
[2219]	validation_0-mlogloss:0.572967
[2220]	validation_0-mlogloss:0.572998
[2221]	validation_0-mlogloss:0.572963
[2222]	validation_0-mlogloss:0.572986
[2223]	validation_0-mlogloss:0.573018
[2224]	valida

[2415]	validation_0-mlogloss:0.570922
[2416]	validation_0-mlogloss:0.570999
[2417]	validation_0-mlogloss:0.57095
[2418]	validation_0-mlogloss:0.570923
[2419]	validation_0-mlogloss:0.570897
[2420]	validation_0-mlogloss:0.570856
[2421]	validation_0-mlogloss:0.570859
[2422]	validation_0-mlogloss:0.570939
[2423]	validation_0-mlogloss:0.570922
[2424]	validation_0-mlogloss:0.570937
[2425]	validation_0-mlogloss:0.570956
[2426]	validation_0-mlogloss:0.570954
[2427]	validation_0-mlogloss:0.570912
[2428]	validation_0-mlogloss:0.571002
[2429]	validation_0-mlogloss:0.571043
[2430]	validation_0-mlogloss:0.571072
[2431]	validation_0-mlogloss:0.571127
[2432]	validation_0-mlogloss:0.571128
[2433]	validation_0-mlogloss:0.571118
[2434]	validation_0-mlogloss:0.571236
[2435]	validation_0-mlogloss:0.571231
[2436]	validation_0-mlogloss:0.57123
[2437]	validation_0-mlogloss:0.571254
[2438]	validation_0-mlogloss:0.571283
[2439]	validation_0-mlogloss:0.571252
[2440]	validation_0-mlogloss:0.571202
[2441]	validat

[2632]	validation_0-mlogloss:0.5691
[2633]	validation_0-mlogloss:0.569115
[2634]	validation_0-mlogloss:0.569067
[2635]	validation_0-mlogloss:0.568985
[2636]	validation_0-mlogloss:0.569
[2637]	validation_0-mlogloss:0.569018
[2638]	validation_0-mlogloss:0.569023
[2639]	validation_0-mlogloss:0.569035
[2640]	validation_0-mlogloss:0.568964
[2641]	validation_0-mlogloss:0.569008
[2642]	validation_0-mlogloss:0.569037
[2643]	validation_0-mlogloss:0.568983
[2644]	validation_0-mlogloss:0.568901
[2645]	validation_0-mlogloss:0.568909
[2646]	validation_0-mlogloss:0.568861
[2647]	validation_0-mlogloss:0.568814
[2648]	validation_0-mlogloss:0.568713
[2649]	validation_0-mlogloss:0.568713
[2650]	validation_0-mlogloss:0.568754
[2651]	validation_0-mlogloss:0.568779
[2652]	validation_0-mlogloss:0.568789
[2653]	validation_0-mlogloss:0.568794
[2654]	validation_0-mlogloss:0.568795
[2655]	validation_0-mlogloss:0.568722
[2656]	validation_0-mlogloss:0.568753
[2657]	validation_0-mlogloss:0.568725
[2658]	validation

[2849]	validation_0-mlogloss:0.568651
[2850]	validation_0-mlogloss:0.568708
[2851]	validation_0-mlogloss:0.568637
[2852]	validation_0-mlogloss:0.568634
[2853]	validation_0-mlogloss:0.568564
[2854]	validation_0-mlogloss:0.568579
[2855]	validation_0-mlogloss:0.568608
[2856]	validation_0-mlogloss:0.568667
[2857]	validation_0-mlogloss:0.568728
[2858]	validation_0-mlogloss:0.568709
[2859]	validation_0-mlogloss:0.568707
[2860]	validation_0-mlogloss:0.568686
[2861]	validation_0-mlogloss:0.568857
[2862]	validation_0-mlogloss:0.568878
[2863]	validation_0-mlogloss:0.568906
[2864]	validation_0-mlogloss:0.568909
[2865]	validation_0-mlogloss:0.568969
[2866]	validation_0-mlogloss:0.568915
[2867]	validation_0-mlogloss:0.568965
[2868]	validation_0-mlogloss:0.568974
[2869]	validation_0-mlogloss:0.569052
[2870]	validation_0-mlogloss:0.569024
[2871]	validation_0-mlogloss:0.569137
[2872]	validation_0-mlogloss:0.569104
[2873]	validation_0-mlogloss:0.569069
[2874]	validation_0-mlogloss:0.568966
[2875]	valid

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [47]:
preds_proba.shape

(3916, 3)

In [45]:
print ("logloss: %0.3f " % log_loss(true_label, preds_proba))
print("Current Accuracy: {0:.3f}".format(accuracy_score(preds_proba,true_label)))

logloss: 0.568 


ValueError: Classification metrics can't handle a mix of continuous-multioutput and multiclass targets

In [21]:
## Loading Glove vectors
embeddings_index = {}
f = open('../../vectors/glove.42B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1917495it [03:07, 10251.27it/s]

Found 1917495 word vectors.


In [22]:
embeddings_index['woman'][0:50]

array([ -1.90770000e-01,   6.50600016e-01,  -3.05849999e-01,
         3.18800002e-01,  -7.07940012e-02,   2.48410001e-01,
        -2.73169994e+00,  -4.37580012e-02,  -5.26859999e-01,
        -7.58560002e-01,  -1.15719996e-02,  -8.15320015e-01,
         6.35939986e-02,  -6.97100013e-02,  -1.55300006e-01,
        -3.90240014e-01,   6.99109972e-01,  -1.29740000e-01,
        -1.41599998e-01,   1.02559999e-01,   1.41190004e-03,
        -5.15209995e-02,   3.72379988e-01,  -6.20259997e-03,
         1.69630006e-01,   3.92159998e-01,  -3.15310001e-01,
        -4.25500005e-01,  -4.13399994e-01,  -8.79890025e-02,
        -4.49970007e-01,   3.94419990e-02,   5.83199978e-01,
         4.32000011e-01,   1.85460001e-01,   6.25000000e-01,
         4.14629988e-02,   5.63820004e-01,  -1.52940005e-01,
        -5.30759990e-02,   2.59259999e-01,   2.45179996e-01,
         1.74899995e-01,  -5.71019985e-02,  -1.26440004e-02,
         2.52829999e-01,   1.42220005e-01,  -1.61740005e-01,
        -1.93450004e-01,

In [23]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stops]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [24]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(x_train)]
xvalid_glove = [sent2vec(x) for x in tqdm(x_test)]

100%|██████████| 3916/3916 [00:01<00:00, 3669.54it/s]


In [25]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [31]:
xvalid_glove.shape[0]/(xtrain_glove.shape[0]+xvalid_glove.shape[0])

0.2000102150263037

In [32]:
## Testing XGBoost with Glove Features
classi = xgb.XGBClassifier(nthread=10, silent=False)
classi.fit(xtrain_glove,y_train)
preds_proba = classi.predict_proba(xvalid_glove)
preds = classi.predict(xvalid_glove)

In [44]:
print("logloss: {}".format(multiclass_logloss(true_label, preds_proba)))
print("Current Accuracy: {0:.3f}".format(accuracy_score(preds,true_label)))

logloss: 0.8641022256384314
Current Accuracy: 0.620


In [38]:
## Testing XGBoost with Glove Features, including early stopping
classi = xgb.XGBClassifier(max_depth=5, n_estimators=500, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
eval_set = [(xvalid_glove,y_test)]

classi.fit(xtrain_glove,y_train, eval_metric = "mlogloss", eval_set=eval_set, verbose=True)
preds_proba = classi.predict_proba(xvalid_glove)
preds = classi.predict(xvalid_glove)

[0]	validation_0-mlogloss:1.083
[1]	validation_0-mlogloss:1.06869
[2]	validation_0-mlogloss:1.05532
[3]	validation_0-mlogloss:1.04284
[4]	validation_0-mlogloss:1.0328
[5]	validation_0-mlogloss:1.0232
[6]	validation_0-mlogloss:1.01433
[7]	validation_0-mlogloss:1.00591
[8]	validation_0-mlogloss:0.998184
[9]	validation_0-mlogloss:0.990696
[10]	validation_0-mlogloss:0.983684
[11]	validation_0-mlogloss:0.976829
[12]	validation_0-mlogloss:0.970942
[13]	validation_0-mlogloss:0.965082
[14]	validation_0-mlogloss:0.959676
[15]	validation_0-mlogloss:0.954636
[16]	validation_0-mlogloss:0.949988
[17]	validation_0-mlogloss:0.945397
[18]	validation_0-mlogloss:0.94142
[19]	validation_0-mlogloss:0.936821
[20]	validation_0-mlogloss:0.932895
[21]	validation_0-mlogloss:0.929154
[22]	validation_0-mlogloss:0.92586
[23]	validation_0-mlogloss:0.9219
[24]	validation_0-mlogloss:0.919157
[25]	validation_0-mlogloss:0.916047
[26]	validation_0-mlogloss:0.91294
[27]	validation_0-mlogloss:0.909921
[28]	validation_0-m

[226]	validation_0-mlogloss:0.780447
[227]	validation_0-mlogloss:0.780298
[228]	validation_0-mlogloss:0.779905
[229]	validation_0-mlogloss:0.780023
[230]	validation_0-mlogloss:0.77998
[231]	validation_0-mlogloss:0.779709
[232]	validation_0-mlogloss:0.779754
[233]	validation_0-mlogloss:0.779441
[234]	validation_0-mlogloss:0.779147
[235]	validation_0-mlogloss:0.778967
[236]	validation_0-mlogloss:0.779066
[237]	validation_0-mlogloss:0.778972
[238]	validation_0-mlogloss:0.77888
[239]	validation_0-mlogloss:0.779058
[240]	validation_0-mlogloss:0.778936
[241]	validation_0-mlogloss:0.778812
[242]	validation_0-mlogloss:0.77865
[243]	validation_0-mlogloss:0.778406
[244]	validation_0-mlogloss:0.778472
[245]	validation_0-mlogloss:0.778063
[246]	validation_0-mlogloss:0.778005
[247]	validation_0-mlogloss:0.777684
[248]	validation_0-mlogloss:0.777099
[249]	validation_0-mlogloss:0.776754
[250]	validation_0-mlogloss:0.776264
[251]	validation_0-mlogloss:0.77609
[252]	validation_0-mlogloss:0.776315
[253]

[449]	validation_0-mlogloss:0.764409
[450]	validation_0-mlogloss:0.764381
[451]	validation_0-mlogloss:0.764268
[452]	validation_0-mlogloss:0.764208
[453]	validation_0-mlogloss:0.763902
[454]	validation_0-mlogloss:0.764007
[455]	validation_0-mlogloss:0.764078
[456]	validation_0-mlogloss:0.76402
[457]	validation_0-mlogloss:0.763829
[458]	validation_0-mlogloss:0.763817
[459]	validation_0-mlogloss:0.764143
[460]	validation_0-mlogloss:0.76433
[461]	validation_0-mlogloss:0.764189
[462]	validation_0-mlogloss:0.764099
[463]	validation_0-mlogloss:0.764424
[464]	validation_0-mlogloss:0.764313
[465]	validation_0-mlogloss:0.764274
[466]	validation_0-mlogloss:0.764365
[467]	validation_0-mlogloss:0.764018
[468]	validation_0-mlogloss:0.763783
[469]	validation_0-mlogloss:0.763854
[470]	validation_0-mlogloss:0.763898
[471]	validation_0-mlogloss:0.763868
[472]	validation_0-mlogloss:0.763741
[473]	validation_0-mlogloss:0.76379
[474]	validation_0-mlogloss:0.76386
[475]	validation_0-mlogloss:0.763956
[476]

In [48]:
print("Log-loss: {0:.3f}".format(multiclass_logloss(true_label, preds_proba)))
print("Current Accuracy: {0:.3f}".format(accuracy_score(preds,true_label)))

Log-loss: 0.831
Current Accuracy: 0.671


## LSTM classifier in Keras using Glove features

In [58]:
## LSTM classifier
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization

In [59]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [60]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(y_train)
yvalid_enc = np_utils.to_categorical(y_test)

In [92]:
# create 3 layer neural net
model = Sequential()
model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.8))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.8))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.8))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [93]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=20, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 15663 samples, validate on 3916 samples
Epoch 1/20
15663/15663 [==============================] - 6s 381us/step - loss: 1.3725 - val_loss: 1.0641
Epoch 2/20
15663/15663 [==============================] - 5s 313us/step - loss: 1.1274 - val_loss: 1.0753
Epoch 3/20
15663/15663 [==============================] - 5s 307us/step - loss: 1.0784 - val_loss: 1.0822
Epoch 4/20
15663/15663 [==============================] - 5s 306us/step - loss: 1.0480 - val_loss: 1.0424
Epoch 5/20
15663/15663 [==============================] - 5s 310us/step - loss: 1.0098 - val_loss: 0.9682
Epoch 6/20
15663/15663 [==============================] - 5s 309us/step - loss: 0.9752 - val_loss: 0.9300
Epoch 7/20
15663/15663 [==============================] - 5s 313us/step - loss: 0.9449 - val_loss: 0.9008
Epoch 8/20
15663/15663 [==============================] - 5s 309us/step - loss: 0.9116 - val_loss: 0.8743
Epoch 9/20
15663/15663 [==============================] - 5s 304us/step - loss: 0.8992 - val_loss: 0.86

In [91]:
preds_proba = model.predict_proba(xvalid_glove)
print("Log-loss: {0:.3f}".format(multiclass_logloss(true_label, preds_proba)))

Log-loss: 1.987


(3916, 3)

## Generate submission

In [ ]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

In [ ]:
def get_txt_proba_response(msg, vectors, model):
    msg_vec = vectors.transform([msg])
#    print(msg_vec)
    pred_prob=model.predict_proba(msg_vec)
    pd_unsorted = pd.DataFrame(
        {'label_encode': model.classes_,
         'label_decode': label_enconder.inverse_transform(model.classes_),
         'pred_proba':  pred_prob[0]})
    
    probas = {
        pd_unsorted.sort_values(['pred_proba'], ascending=False,axis=0).iloc[0][0]:
        pd_unsorted.sort_values(['pred_proba'], ascending=False,axis=0).iloc[0][2],
        pd_unsorted.sort_values(['pred_proba'], ascending=False,axis=0).iloc[1][0]:
        pd_unsorted.sort_values(['pred_proba'], ascending=False,axis=0).iloc[1][2],
         pd_unsorted.sort_values(['pred_proba'], ascending=False,axis=0).iloc[2][0]:
         pd_unsorted.sort_values(['pred_proba'], ascending=False,axis=0).iloc[2][2]}
    
    return probas

In [ ]:
for i in range(len(test)):
    reply=get_txt_proba_response(transformText(test['text'][i]),features_2, model_sgd_2)
    my_sub.loc[i] = [test['id'][i], reply['EAP'], reply['HPL'], reply['MWS']]

In [ ]:
my_sub

In [ ]:
my_sub.to_csv('roberto.csv',index=False)

## Testing text classification with PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class  CNN_Text(nn.Module):
    
    def __init__(self, args):
        super(CNN_Text,self).__init__()
        self.args = args
        
        V = args.embed_num
        D = args.embed_dim
        C = args.class_num
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes

        self.embed = nn.Embedding(V, D)
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        self.dropout = nn.Dropout(args.dropout)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3) #(N,Co,W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embed(x) # (N,W,D)
        if self.args.static:
            x = Variable(x)
        x = x.unsqueeze(1) 
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = torch.cat(x, 1)
        x = self.dropout(x) # (N,len(Ks)*Co)
        logit = self.fc1(x) # (N,C)
        return logit